<html>
<head>
	<title></title>
</head>
<body>
<h1 style="text-align: center;line-height: 120%;">ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ВЫСШЕГО ОБРАЗОВАНИЯ НАЦИОНАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ УНИВЕРСИТЕТ «МОСКОВСКИЙ ЭНЕРГЕТИЧЕСКИЙ ИНСТИТУТ» Кафедра «РЗиАЭ»</h1>
<h2 style="text-align: center;line-height: 20%;">Лабораторная работа №3</h2>

<h3 style="text-align: center;line-height: 80%;"><em>Симплекс-метод</em></h3>

<p style="text-align: right;line-height: 40%;">Выполнил: студент группы Э-13м-19</p>
<p style="text-align: right;line-height: 40%;">Максимов Р.С.</p>

<p style="text-align: right;line-height: 40%;">Принял: Грачева Н. П.</p>
<h4 style="text-align: center;line-height: 100%;"><em> Москва, 2020</em></h3>
</body>
</html>

In [1]:
import numpy as np
import time
import random 
import scipy
from scipy import spatial
import math

Для реализации трех групп товаров коммерческое предприятие располагает тремя видами ограниченных материально-денежных ресурсов в количестве b1, b2, b3, единиц. При этом для продажи 1 группы товаров на 1 тыс. руб. товарооборота расходуется ресурса первого вида в количестве a11 единиц, ресурса второго вида в количестве a21 единиц, ресурса третьего вида в количестве a31 единиц. Для продажи 2 и 3 групп товаров на 1 тыс. руб. товарооборота расходуется соответственно ресурса первого вида в количестве a12, a13 единиц, ресурсов второго вида в количестве a22, a23 единиц, ресурсов третьего вида в количестве a32, a33 единиц. Прибыль от продажи трех групп товаров на 1 тыс. руб. товарооборота составляет соответственно c1, c2, c3 тыс. руб

Необходимо, определить плановый объем и структуру товарооборота так, чтобы прибыль торгового предприятия была максимальной.

In [86]:
def search_min(f):
    minimum = 9*max(f)
#     print(f)
    for i in range(len(f)):        
        if f[i] < minimum and (round(f[i],3) < 0.):
            minimum = f[i]
#             print(minimum)
    return minimum
        
        

In [93]:
def simplex_method(a,b,F):
    b = np.array(b)
    F = np.array(F)
    F = F*-1
    a.extend([F])
    allTable = np.array(a)    
#     print(allTable) 
    y = np.eye(len(b),len(b)-1)
    b = np.hstack((b,[0]))
    b_column = b.reshape(len(b),1)
    y = np.eye(len(b),len(b)-1) #для добавления,формирования конечной матрицы
    g = np.hstack((allTable, y)) 
    end_table = np.hstack((g, b_column)) # матрица
    last_row = end_table.shape[0]-1
    last_colomn = end_table.shape[1]-1
    
    min_number_row = search_min(end_table[last_row,:])
    print(min_number_row)
    i,  = np.where(end_table[last_row,:] == min_number_row)
#     print(type(i))
    print(i)
    i = int(i)
    
    bases = np.arange(len(F)+1,last_colomn+1)
#     print(bases)
    iks = np.arange(1,len(F)+1)
    
    while min_number_row < 0:
        print('Актуальная матрица = ', end_table)
        #находим индекс самого минимального и отрицательного элемента
        i,  = np.where(end_table[last_row,:] == min_number_row)
        i = int(i)
        print("Индекс минимального элемента в последней строке = ", i)
        #нулевой разрешающий столбец
        column = np.zeros((len(b),1))
        #формируем разрешающий стобец
        for j in range(len(b)):
            column[j,0] = np.abs(end_table[j,last_colomn] / end_table[j,i])
        print("Дополнительный столбец = ", column)
        #находим минимальное число из разрешающего столбца
        min_number_column = min(column[:len(column)-1,0]) #все кроме последнего элемента(он ноль)
        print("Мин. число из дополнительного столбца = ", min_number_column)
        #находим индекс, таким образом, находим разрешающуюю строку
        k,  = np.where(column[:,0] == min_number_column)
        print("Индекс минимального элемента в дополнительном столбце = ", k)
        k = int(k)
        #Изменяем текущий базис
        bases[k] = i + 1
        #находим разрешающий коэф
        enable_coef = (end_table[k,i])
#         print('Разрешающий коэф = ',enable_coef)
        #делим разрещающую строку на разрешающий коэф.
        end_table[k,:] = end_table[k,:] / enable_coef
        #формируем нулевую разрешающую строку
        enable_row = end_table[k,:]
        for m in range(end_table.shape[0]):
            if not(m == k):
                end_table[m,:] =  end_table[m,:] - enable_row * end_table[m,i]        
        end_table = np.round_(end_table, decimals=2, out=None)
        min_number_row = min(end_table[last_row,:])
#     print(bases)
    print('Конечная матрица = ', end_table)
    #вывод значений, формирование базисов
    bases_X = np.arange(0,len(F))*0 #лист из нулевых базисов
    X = [0.]*len(F) #лист из нулевых иксов
    #
    for count in range(len(b)-1):       
        number_basis = bases[count]   #актуальный базис в цикле из списка всех конечных базисов     
        if number_basis in iks: 
            index_b, = np.where(bases == number_basis) #если номер базиса равен искомому номеру икса, то  
            index_b = int(index_b) #находим индекс
            X[number_basis-1] = float(end_table[index_b,last_colomn]) 

        
#     print(X)
    maxFunc = end_table[last_row,last_colomn]
#     print(maxFunc)
    return X,maxFunc

In [94]:
a = [[32, 11, 4],[5, 16, 32],[21, 12, 5]]
b = [520, 550, 90]
c = [5, 3, 0]
F = c
x,f = simplex_method(a,b,c)
print('Оптимальные X = ', x)
print('Значение функции при оптимальных X = ',f)

-5.0
[0]
Актуальная матрица =  [[ 32.  11.   4.   1.   0.   0. 520.]
 [  5.  16.  32.   0.   1.   0. 550.]
 [ 21.  12.   5.   0.   0.   1.  90.]
 [ -5.  -3.   0.   0.   0.   0.   0.]]
Индекс минимального элемента в последней строке =  0
Дополнительный столбец =  [[ 16.25      ]
 [110.        ]
 [  4.28571429]
 [  0.        ]]
Мин. число из дополнительного столбца =  4.285714285714286
Индекс минимального элемента в дополнительном столбце =  [2]
Актуальная матрица =  [[ 0.0000e+00 -7.2900e+00 -3.6200e+00  1.0000e+00  0.0000e+00 -1.5200e+00
   3.8286e+02]
 [ 0.0000e+00  1.3140e+01  3.0810e+01  0.0000e+00  1.0000e+00 -2.4000e-01
   5.2857e+02]
 [ 1.0000e+00  5.7000e-01  2.4000e-01  0.0000e+00  0.0000e+00  5.0000e-02
   4.2900e+00]
 [ 0.0000e+00 -1.4000e-01  1.1900e+00  0.0000e+00  0.0000e+00  2.4000e-01
   2.1430e+01]]
Индекс минимального элемента в последней строке =  1
Дополнительный столбец =  [[ 52.51851852]
 [ 40.2260274 ]
 [  7.52631579]
 [153.07142857]]
Мин. число из дополнительного

In [85]:
a = [[32, 11, 4, 2],[5, 16, 32,2],[21, 12, 5, 0],[15, 12, 13,10],[10, 9, 0,8]]
b = [520, 550, 90, 110, 300]
c = [5, 3, 7, 8]
x,f = simplex_method(a,b,c)
print('Оптимальные X = ', x)
print('Значение функции при оптимальных X = ',f)

-8.0
[3]
Актуальная матрица =  [[ 32.  11.   4.   2.   1.   0.   0.   0.   0. 520.]
 [  5.  16.  32.   2.   0.   1.   0.   0.   0. 550.]
 [ 21.  12.   5.   0.   0.   0.   1.   0.   0.  90.]
 [ 15.  12.  13.  10.   0.   0.   0.   1.   0. 110.]
 [ 10.   9.   0.   8.   0.   0.   0.   0.   1. 300.]
 [ -5.  -3.  -7.  -8.   0.   0.   0.   0.   0.   0.]]
Индекс минимального элемента в последней строке =  3
Дополнительный столбец =  [[260. ]
 [275. ]
 [  inf]
 [ 11. ]
 [ 37.5]
 [ -0. ]]
Мин. число из дополнительного столбца =  11.0
Индекс минимального элемента в дополнительном столбце =  [3]
Конечная матрица =  [[ 2.90e+01  8.60e+00  1.40e+00  0.00e+00  1.00e+00  0.00e+00  0.00e+00
  -2.00e-01  0.00e+00  4.98e+02]
 [ 2.00e+00  1.36e+01  2.94e+01  0.00e+00  0.00e+00  1.00e+00  0.00e+00
  -2.00e-01  0.00e+00  5.28e+02]
 [ 2.10e+01  1.20e+01  5.00e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00
   0.00e+00  0.00e+00  9.00e+01]
 [ 1.50e+00  1.20e+00  1.30e+00  1.00e+00  0.00e+00  0.00e+00  0.00e+00
  

C:\Users\maksimov\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: divide by zero encountered in double_scalars
